Normalisation par lots

In [27]:
import tensorflow as tf

n_inputs = 28 * 28  
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_inputs), name="X")
training = tf.compat.v1.placeholder_with_default(False, shape=(), name='training')

from functools import partial

my_batch_norm_layer = partial(tf.compat.v1.layers.batch_normalization, training=training, momentum=0.9)

hidden1 = tf.compat.v1.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.compat.v1.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.compat.v1.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

/tmp/ipykernel_37700/1542062223.py:17: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  hidden1 = tf.compat.v1.layers.dense(X, n_hidden1, name="hidden1")


ValueError: Variable hidden1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?

In [21]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [22]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [26]:
import tensorflow as tf
from functools import partial

batch_norm_momentum = 0.9
n_inputs = 28 * 28  
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.compat.v1.placeholder(tf.int32, shape=(None), name="y")
training = tf.compat.v1.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("dnn"):
    he_init = tf.keras.initializers.VarianceScaling()

    my_batch_norm_layer = partial(
        tf.keras.layers.BatchNormalization,
        momentum=batch_norm_momentum)

    my_dense_layer = partial(
        tf.keras.layers.Dense,
        kernel_initializer=he_init)

    hidden1 = my_dense_layer(n_hidden1, name="hidden1")(X)
    bn1 = tf.nn.elu(my_batch_norm_layer()(hidden1, training=training))
    hidden2 = my_dense_layer(n_hidden2, name="hidden2")(bn1)
    bn2 = tf.nn.elu(my_batch_norm_layer()(hidden2, training=training))
    logits_before_bn = my_dense_layer(n_outputs, name="outputs")(bn2)
    logits = my_batch_norm_layer()(logits_before_bn, training=training)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

/home/honor/Documents/MachineLearning/env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


TypeError: Input 'predictions' of 'InTopKV2' Op has type int32 that does not match expected type of float32.

In [23]:
import numpy as np

n_epochs = 20
batch_size = 200

X_train = train_images.reshape(-1, 28*28) / 255.0
y_train = train_labels

extra_update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

NameError: name 'y' is not defined